In [26]:
!pip install opencv-python numpy


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Motion Analysis:** *Quantify the average motion using Optical Flow.*

In [28]:
import cv2
import numpy as np

def analyze_motion(video_path: str) -> float:
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return 0.0

    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Cannot read first frame.")
        return 0.0

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    motion_magnitudes = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow between prev_gray and gray
        flow = cv2.calcOpticalFlowFarneback(
            prev_gray, gray, None,
            pyr_scale=0.5, levels=3, winsize=15,
            iterations=3, poly_n=5, poly_sigma=1.2, flags=0
        )

        magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        motion_magnitudes.append(np.mean(magnitude))

        prev_gray = gray

    cap.release()

    avg_motion = float(np.mean(motion_magnitudes)) if motion_magnitudes else 0.0
    return avg_motion




#**Text Detection (OCR):** *Determine a text_present_ratio or extract keywords.*

In [29]:
!pip install easyocr


In [30]:
import cv2
import easyocr
import re
from collections import Counter
import torch  # to check GPU availability


def clean_words(text: str):
    """
    Basic cleanup:
      - keep only alphabetic tokens
      - lowercase
      - length >= 3
    """
    words = re.findall(r"[A-Za-z]+", text.lower())
    filtered = [w for w in words if len(w) >= 3]
    return filtered


def analyze_text_ocr_easyocr(
    video_path: str,
    ocr_frame_step: int = 10,
    languages=None,
    min_conf: float = 0.5,
) -> dict:
    """
    Analyze a video using EasyOCR:
      - text_present_ratio: fraction of sampled frames that contain text
      - keywords: cleaned list of unique words detected by OCR
      - Automatically uses GPU if available
    """
    if languages is None:
        languages = ['en']

    # 🔍 Check for GPU
    use_gpu = torch.cuda.is_available()
    print(f"🔧 Using GPU: {use_gpu}")

    # Initialize EasyOCR reader
    reader = easyocr.Reader(languages, gpu=use_gpu)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return {
            "file": video_path,
            "text_present_ratio": 0.0,
            "keywords": [],
        }

    ocr_total_frames = 0
    ocr_text_frames = 0
    all_words = []

    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Sample every Nth frame for OCR
        if frame_idx % ocr_frame_step == 0:
            ocr_total_frames += 1

            # results: list of [bbox, text, confidence]
            results = reader.readtext(frame, detail=True)

            # Keep only high-confidence texts
            valid_texts = [
                text for (bbox, text, conf) in results
                if conf >= min_conf and text.strip()
            ]

            if valid_texts:
                ocr_text_frames += 1
                for t in valid_texts:
                    all_words.extend(clean_words(t))

        frame_idx += 1

    cap.release()

    text_present_ratio = (
        ocr_text_frames / ocr_total_frames if ocr_total_frames > 0 else 0.0
    )

    keyword_counter = Counter(all_words)
    keywords = list(keyword_counter.keys())

    return {
        "file": video_path,
        "text_present_ratio": float(text_present_ratio),
        "keywords": keywords,
    }


# if __name__ == "__main__":
#     video_path = "/content/drive/MyDrive/Test_Video/test_video_subtitle.mp4"
#     features = analyze_text_ocr_easyocr(video_path)
#     print(features)


# **Shot Cut Detection:** *Calculate the number of "hard cuts."*

In [31]:
import cv2
import numpy as np


def detect_hard_cuts(
    video_path: str,
    diff_threshold: float = 30.0,
    min_scene_length: int = 5
) -> dict:
    """
    Detect hard cuts (abrupt shot changes) in a video.

    A "hard cut" is detected when the mean pixel difference between
    consecutive frames exceeds `diff_threshold`.

    :param video_path: path to the video file
    :param diff_threshold: threshold on mean abs diff to decide a cut (0–255)
    :param min_scene_length: minimum number of frames between cuts
                             (to avoid counting tiny flickers as multiple cuts)
    :return: dict with number of cuts and frame indices of cuts
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return {
            "file": video_path,
            "hard_cuts": 0,
            "cut_frames": [],
        }

    # Read first frame
    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Cannot read first frame.")
        cap.release()
        return {
            "file": video_path,
            "hard_cuts": 0,
            "cut_frames": [],
        }

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    hard_cuts = 0
    cut_frames = []
    frame_idx = 1          # we already read frame 0
    last_cut_frame = -min_scene_length  # so first cut is not blocked

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Mean absolute difference between consecutive frames
        diff = cv2.absdiff(gray, prev_gray)
        diff_score = float(np.mean(diff))  # 0–255

        # Decide if this is a hard cut
        if diff_score > diff_threshold and (frame_idx - last_cut_frame) >= min_scene_length:
            hard_cuts += 1
            cut_frames.append(frame_idx)
            last_cut_frame = frame_idx

        prev_gray = gray
        frame_idx += 1

    cap.release()

    return {
        "file": video_path,
        "hard_cuts": hard_cuts,
        "cut_frames": cut_frames,
    }


# if __name__ == "__main__":
#     video_path = "/content/drive/MyDrive/Test_Video/hard_cut.mp4"
#     result = detect_hard_cuts(video_path)
#     print(result)


# **Object vs. Person Dominance:** *Calculate a ratio of people versus objects detected using a pre-trained model YOLO.*

In [32]:
!pip install ultralytics


In [33]:
import cv2
import numpy as np
from ultralytics import YOLO


def analyze_person_object_dominance(
    video_path: str,
    model_path: str = "yolov8x.pt",
    det_frame_step: int = 10,
    conf_thres: float = 0.5,
) -> dict:
    """
    Analyze a video using YOLO to estimate person vs object dominance.

    - person_detections: total person boxes across sampled frames
    - object_detections: total non-person boxes
    - person_ratio: person_detections / (person + object)
    - object_ratio: object_detections / (person + object)
    - person_to_object_ratio: person_detections / object_detections

    :param video_path: path to the video file
    :param model_path: YOLO model weights (e.g. 'yolov8n.pt')
    :param det_frame_step: run detection on every Nth frame
    :param conf_thres: minimum confidence for counting a detection
    :return: dict with counts and ratios
    """

    # Load YOLO model (pretrained on COCO by default)
    model = YOLO(model_path)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return {
            "file": video_path,
            "person_detections": 0,
            "object_detections": 0,
            "person_ratio": 0.0,
            "object_ratio": 0.0,
            "person_to_object_ratio": None,
        }

    person_detections = 0
    object_detections = 0
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Only run YOLO on every Nth frame for speed
        if frame_idx % det_frame_step == 0:
            # Run YOLO; results is a list of Results objects
            results = model(frame, verbose=False)

            if not results:
                frame_idx += 1
                continue

            result = results[0]  # single image

            boxes = result.boxes
            if boxes is not None and len(boxes) > 0:
                for box in boxes:
                    cls_id = int(box.cls[0])
                    conf = float(box.conf[0])

                    if conf < conf_thres:
                        continue

                    class_name = model.names[cls_id]
                    if class_name == "person":
                        person_detections += 1
                    else:
                        object_detections += 1

        frame_idx += 1

    cap.release()

    total_detections = person_detections + object_detections
    if total_detections > 0:
        person_ratio = person_detections / total_detections
        object_ratio = object_detections / total_detections
    else:
        person_ratio = 0.0
        object_ratio = 0.0

    if object_detections > 0:
        person_to_object_ratio = person_detections / object_detections
    else:
        # If there are no objects but people exist, dominance is effectively "infinite"
        person_to_object_ratio = None  # or float("inf") if you prefer

    return {
        "file": video_path,
        "person_detections": int(person_detections),
        "object_detections": int(object_detections),
        "person_ratio": float(person_ratio),
        "object_ratio": float(object_ratio),
        "person_to_object_ratio": person_to_object_ratio,
    }


In [34]:
# if __name__ == "__main__":
#     video_path = "/content/drive/MyDrive/Test_Video/hard_cut_test.mp4"
#     features = analyze_person_object_dominance(video_path)
#     print(features)

In [35]:
import json

def extract_video_features(video_path: str) -> dict:
    """
    Run all feature extractors on a video and return a unified feature dict.

    Features:
      - hard_cuts, cut_frames
      - average_motion
      - text_present_ratio, keywords
      - person/object dominance metrics
    """
    # ---- 1. Shot cut detection ----
    cut_info = detect_hard_cuts(video_path)
    # cut_info -> { "file": ..., "hard_cuts": int, "cut_frames": [...] }

    # ---- 2. Motion analysis ----
    avg_motion = analyze_motion(video_path)
    # avg_motion -> float

    # ---- 3. Text detection (OCR via EasyOCR) ----
    ocr_info = analyze_text_ocr_easyocr(
        video_path,
        ocr_frame_step=10,   # sample every 10th frame for speed
        languages=['en'],    # adjust if you have other languages
        min_conf=0.5,
    )
    # ocr_info -> { "file": ..., "text_present_ratio": float, "keywords": [...] }

    # ---- 4. Person vs object dominance (YOLO) ----
    dominance_info = analyze_person_object_dominance(
        video_path,
        model_path="yolov8n.pt",  # or yolov8s.pt if you want more accuracy
        det_frame_step=10,
        conf_thres=0.5,
    )
    # dominance_info -> { "file": ..., "person_detections": ..., "object_detections": ..., ... }

    # ---- 5. Combine everything in ONE structured dict ----
    features = {
        "file": video_path,

        # Shot cuts
        "hard_cuts": cut_info.get("hard_cuts", 0),
        "cut_frames": cut_info.get("cut_frames", []),

        # Motion
        "average_motion": float(avg_motion),

        # OCR / text presence
        "text_present_ratio": float(ocr_info.get("text_present_ratio", 0.0)),
        "keywords": ocr_info.get("keywords", []),

        # Person vs object dominance
        "person_detections": int(dominance_info.get("person_detections", 0)),
        "object_detections": int(dominance_info.get("object_detections", 0)),
        "avg_persons_per_sampled_frame": float(
            dominance_info.get("avg_persons_per_sampled_frame", 0.0)
        ),
        "person_frame_ratio": float(
            dominance_info.get("person_frame_ratio", 0.0)
        ),
        "person_ratio": float(dominance_info.get("person_ratio", 0.0)),
        "object_ratio": float(dominance_info.get("object_ratio", 0.0)),
    }

    return features


In [36]:
if __name__ == "__main__":
    video_path = "/content/drive/MyDrive/Test_Video/hard_cut_test.mp4"

    features = extract_video_features(video_path)
    print("Extracted features:\n", features)

    # Save to JSON file
    output_path = "/content/drive/MyDrive/Test_Video/video_features.json"
    with open(output_path, "w") as f:
        json.dump(features, f, indent=2)

    print(f"\nFeatures saved to: {output_path}")


🔧 Using GPU: True
Extracted features:
 {'file': '/content/drive/MyDrive/Test_Video/hard_cut_test.mp4', 'hard_cuts': 35, 'cut_frames': [22, 54, 70, 89, 119, 140, 157, 164, 169, 185, 203, 225, 251, 273, 280, 286, 291, 304, 311, 337, 366, 386, 404, 421, 449, 474, 505, 521, 543, 565, 602, 628, 656, 690, 720], 'average_motion': 2.1984872817993164, 'text_present_ratio': 0.3424657534246575, 'keywords': ['superi', 'super', 'gund', 'cornchips', 'iguperi', 'cod', 'tasty', 'cord', 'sweet', 'favor', 'crunch', 'prifood', 'hips', 'alouritq', 'asty', 'corn', 'flavor', 'unc', 'cru', 'try', 'them', 'pasalubong', 'style', 'crackling', 'cheese', 'not', 'walght', 'lechon', 'kawali', 'special', 'chip', 'charon', 'fiavor', 'kowali'], 'person_detections': 97, 'object_detections': 20, 'avg_persons_per_sampled_frame': 0.0, 'person_frame_ratio': 0.0, 'person_ratio': 0.8290598290598291, 'object_ratio': 0.17094017094017094}

Features saved to: /content/drive/MyDrive/Test_Video/video_features.json
